In [1]:
from interpret import show
from interpret.blackbox import ShapKernel
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())
from IoU import IoU_value
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow import keras
from interpret import show
from interpret.blackbox import LimeTabular
import pandas as pd
import numpy as np
import xgboost

In [4]:
def normalize(x):
    if x.max() == x.min():
        return x
    else:
        result = (x-x.min())/(x.max()-x.min())
        return result

# 在生成的数据集中验证SHAP

In [2]:
df = pd.read_csv("train/train_data_0.csv")  #10000x100数据，不包含Target
df

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature90,feature91,feature92,feature93,feature94,feature95,feature96,feature97,feature98,feature99
0,84.071517,15,95,37,109.222187,41.054423,14,12.635977,14.010126,70,...,26,56,32,30,30,65,36.400093,55,44,11.299354
1,64.376450,12,39,40,108.267721,58.627029,46,80.431019,3.598875,70,...,28,38,69,28,10,44,32.313545,30,45,10.109159
2,61.747596,36,58,7,51.315514,10.865075,53,43.284407,52.822478,70,...,20,45,62,43,29,48,35.294801,27,70,11.926330
3,108.096804,32,73,59,40.416383,12.580249,70,19.908695,59.408556,70,...,26,7,57,68,3,42,37.391035,47,71,10.819648
4,93.659688,14,45,30,85.312636,47.390543,81,30.780931,44.901262,71,...,30,10,46,42,3,42,32.743209,42,47,11.301096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,46.443361,29,59,22,41.987071,57.629751,39,78.792761,4.834577,71,...,26,63,38,66,16,62,36.587602,31,50,7.942675
9996,101.995631,23,57,16,100.037233,41.170912,25,32.059358,31.253103,71,...,29,57,44,52,7,62,37.806850,31,70,11.128170
9997,79.100888,42,36,44,100.913728,59.806790,17,69.167360,47.454068,70,...,22,60,46,4,32,62,32.413842,52,59,10.260244
9998,97.844526,31,41,46,52.108113,51.004928,65,39.811859,36.990671,70,...,25,20,42,4,19,47,36.201993,27,54,8.643205


# 线性模型解释

$ y = 5 \times x_0 + 1 \times x_1 + 2 \times x_3$ 

In [5]:
df["label"] = df.feature0 *1 + df.feature1 * 1 + df.feature3*1
df["label"] = (normalize(df["label"])>0.5).astype(int)
df

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature91,feature92,feature93,feature94,feature95,feature96,feature97,feature98,feature99,label
0,84.071517,15,95,37,109.222187,41.054423,14,12.635977,14.010126,70,...,56,32,30,30,65,36.400093,55,44,11.299354,0
1,64.376450,12,39,40,108.267721,58.627029,46,80.431019,3.598875,70,...,38,69,28,10,44,32.313545,30,45,10.109159,0
2,61.747596,36,58,7,51.315514,10.865075,53,43.284407,52.822478,70,...,45,62,43,29,48,35.294801,27,70,11.926330,0
3,108.096804,32,73,59,40.416383,12.580249,70,19.908695,59.408556,70,...,7,57,68,3,42,37.391035,47,71,10.819648,1
4,93.659688,14,45,30,85.312636,47.390543,81,30.780931,44.901262,71,...,10,46,42,3,42,32.743209,42,47,11.301096,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,46.443361,29,59,22,41.987071,57.629751,39,78.792761,4.834577,71,...,63,38,66,16,62,36.587602,31,50,7.942675,0
9996,101.995631,23,57,16,100.037233,41.170912,25,32.059358,31.253103,71,...,57,44,52,7,62,37.806850,31,70,11.128170,1
9997,79.100888,42,36,44,100.913728,59.806790,17,69.167360,47.454068,70,...,60,46,4,32,62,32.413842,52,59,10.260244,1
9998,97.844526,31,41,46,52.108113,51.004928,65,39.811859,36.990671,70,...,20,42,4,19,47,36.201993,27,54,8.643205,1


In [6]:
data = df.iloc[:,:-1]
label = df.iloc[:,-1]
X_train,X_test, y_train,y_test = train_test_split(data,label,test_size = 0.3)

In [11]:
xgb_clf = xgboost.XGBClassifier()
xgb_clf.fit(X_train,y_train)
shap = ShapKernel(predict_fn=xgb_clf.predict_proba, data=X_train,n_jobs=2,)  # 多进程用多了内存容易爆
shap_local = shap.explain_local(X_test[:5], y_test[:5])
show(shap_local)

[20:20:18] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Using 7000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


MemoryError: Unable to allocate 11.7 GiB for an array with shape (2244, 700000) and data type float64